## SICP 习题 （2.56）解题总结：求导规则的增加

SICP 习题 2.56开始又进入普通程序员苦恼的领域，关于导数。

很多程序员可能学过导数，但是忘了！

有更多的程序员可能就没有学习过导数，比如开过课就没怎么听那种。

面对这种题，和之前一样有两种策略，一种是抛开数学概念直接从逻辑角度去理解，这种方法马上可以干活，但是心里面一直都会有点虚；另一种方法就是去补习一下导数的概念，这种方法可以深入理解题目，对以后学习其它计算机领域的知识也起很大作用，比如神经网络的理解就需要对偏导数的概念有清晰的认知，但是这种方法有个极大的弊端，那就是你去翻开数学书或者数学网站的时候你就放弃了。。。。。



回到题目上，这一节的内容主要是关于求导规则，希望用语言的方式实现求导规则。

那首先要看看基本的求导规则：

$$ \frac {dc} {dx} = 0 $$ 

$$ \frac {dx} {dx} = 1 $$ 

$$ \frac {d(u+v)} {dx} = \frac {du} {dx} + \frac {dv} {dx} $$ 

$$ \frac {d(uv)} {dx} = u \left( {\frac {dv} {dx}} \right)  + v \left( {\frac {du} {dx}} \right)$$ 



可以看到这里的求值规则和我们之前考虑的函数求值规则是很类似的，对复杂的求导公式，可以分别对其中的不同部分进行展开，展开成基本的可以直接求值的部分，然后对基本部分进行求值，不断归约，最后得出结果。

要完成习题需要先按照书上的介绍先把基础部分实现了，大部分都可以抄代码.

实现之前可以先看看deriv函数的实现，先看看整体思路，这样对理解单个基础组件很有帮助

In [11]:
(define (deriv exp var)
  (cond ((number? exp) 0)
	((variable? exp)
	 (if (same-variable? exp var) 1 0))
	((sum? exp)
	 (make-sum (deriv (addend exp) var)
		   (deriv (augend exp) var)))
	((product? exp)
	 (make-sum
	  (make-product (multiplier exp)
			(deriv (multiplicand exp) var))
	  (make-product (deriv (multiplier exp) var)
			(multiplicand exp))))
	((exponentiation? exp)
	 (let ((n (exponent exp))
	       (u (base exp)))
	   (make-product
	    n
	    (make-product
	     (make-exponentiation
	      u
	      (- n 1))
	     (deriv u var)))))
	(else 
	 (error "unkonwn expression type -- DERIV" exp))))

以上代码的主要思路就是按求导规则做判断，如果表达式是个数字，就返回0， 如果表达式和求导变量是一个变量，就返回1，如果表达式是个加法，就递归调用derive，然后将结果相加，如果表达式是个乘法，就对因子分别求导，交叉相乘。

为了实现以上的判断，先要实现各种判断函数：

In [12]:
(define (variable? x) (symbol? x))

In [15]:
(variable? '1)

#f

In [14]:
(variable? 'a)

#t

In [16]:
(define (same-variable? v1 v2)
  (and (variable? v1) (variable? v2) (eq? v1 v2)))

In [17]:
(same-variable? 'a 'a)

#t

In [18]:
(same-variable? 'a '1)

#f

In [19]:
(define (make-sum-original a1 a2) (list '+ a1 a2))

In [20]:
(make-sum-original '1 '2)

(+ 1 2)

上面是构建一个和式，不过这个和式没有做简化，1 + 2不就是3嘛，应该返回3才是，

于是有下面这个加强版的(make-sum)

In [24]:
(define (=number? exp num)
  (and (number? exp) (= exp num)))

(define (make-sum a1 a2)
  (cond ((=number? a1 0) a2)
	((=number? a2 0) a1)
	((and (number? a1) (number? a2)) (+ a1 a2))
	(else 
	 (list '+ a1 a2))))

In [25]:
(make-sum '1 '2)

3

乘式也做类似的处理：

In [26]:
(define (make-product-original m1 m2) (list '* m1 m2))

(define (make-product m1 m2)
  (cond ((or (=number? m1 0) (=number? m2 0)) 0)
	((=number? m1 1) m2)
	((=number? m2 1) m1)
	((and (number? m1) (number? m2)) (* m1 m2))
	(else (list '* m1 m2))))

In [27]:
(make-product '3 '45)

135

In [28]:
(make-product 'a '45)

(* a 45)

然后式对加式的判断和处理，通过判断表达式的第一个参数是不是`+`来确定表达式是不是加式。

然后通过取表达式第二个参数取得加法的第一个数

通过取表达式的第三个参数去的加法的第二个数，不过。。。。， 如果是`(+ x y z)`者中形式就有点麻烦，用`(caddr)`可以取第三个参数，但是把后面的z漏掉了，所有就需要判断一下后面还有没有参数，有后面的参数的话就把第三个参数和以后的所有参数变成一个新的加式，就是把`(+ x y z)` 当作 `'(+ x (+ y z))`

当我们处理了这个问题以后其实就把题目2.57也解决了。

In [40]:
(define (sum? x)
  (and (pair? x) (eq? (car x) '+)))

(define (addend s) (cadr s))

(define (augend-original s) (cddr s))

(define (augend s) 
   (if (null? (cdddr s)) 
       (caddr s) 
       (cons '+ (cddr s))))

然后是乘式的处理，方法和加式的处理几乎一样，大家也可以想想是不是可以抽象出更高阶的方法生成加式处理函数和乘式处理函数

In [42]:
(define (product? x)
  (and (pair? x) (eq? (car x) '*)))

(define (multiplier p) (cadr p))

(define (multiplicand-original p) (caddr p))


 (define (multiplicand p) 
   (if (null? (cdddr p)) 
       (caddr p) 
       (cons '* (cddr p)))) 


到这里就可以先测试一下现在的求导函数了：

In [43]:
(deriv '(+ x 3) 'x)

1

In [44]:
(deriv '(* x y) 'x)

y

然后就到了题目要求的增加求导规则了，题目要求增加指数计算的求导。

基本上就是参考加式处理和乘式处理，有(make-exponentitation)用于生成指数表达，然后(exponentiation?)用于判断一个表达式是不是指数式，然后还有获取表达式各元素的方法。

In [46]:
(define (make-exponentiation base exponent)
  (cond ((= exponent 0)
	 1)
	((= exponent 1)
	 base)
	(else
	 (list '** base exponent))))

(define (exponentiation? x)
  (and (pair? x)
       (eq? (car x) '**)))

(define (base exp)
  (cadr exp))

(define (exponent exp)
  (caddr exp))

最后在deriv函数里加上对指数的处理：

In [47]:
(define (deriv exp var)
  (cond ((number? exp) 0)
	((variable? exp)
	 (if (same-variable? exp var) 1 0))
	((sum? exp)
	 (make-sum (deriv (addend exp) var)
		   (deriv (augend exp) var)))
	((product? exp)
	 (make-sum
	  (make-product (multiplier exp)
			(deriv (multiplicand exp) var))
	  (make-product (deriv (multiplier exp) var)
			(multiplicand exp))))
	((exponentiation? exp)
	 (let ((n (exponent exp))
	       (u (base exp)))
	   (make-product
	    n
	    (make-product
	     (make-exponentiation
	      u
	      (- n 1))
	     (deriv u var)))))
	(else 
	 (error "unkonwn expression type -- DERIV" exp))))


(define (start-base-test-2-56)
  (display (deriv '(+ x 3) 'x)) (newline)

  (display (deriv '(* x y) 'x)) (newline)

  (display (deriv '(* (* x y) (+ x 3)) 'x))(newline)) 

(define (start-test-2-56)
  (display (deriv '(** x 6) 'x)) (newline))

In [48]:
(start-base-test-2-56)

1
y
(+ (* x y) (* y (+ x 3)))


In [49]:
(start-test-2-56)

(* 6 (** x 5))
